<a href="https://colab.research.google.com/github/vmt311/Finetune-LLM/blob/main/FinetuneLLM_guessPrice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Python >=3.10
!pip install -U "transformers>=4.44" "datasets>=2.20" "accelerate>=0.33" "peft>=0.13" \
  "trl>=0.9" "bitsandbytes>=0.43" "evaluate" "scipy" "sentencepiece"

In [ ]:
from google.colab import userdata
from huggingface_hub import login

hf_token = userdata.get("HF_TOKEN")

login(token=hf_token)

In [ ]:
from datasets import load_dataset

dataset = load_dataset("vmt311/bookpricer-data")

dataset

In [ ]:
train=dataset["train"]
test=dataset["test"]

In [ ]:
train[1]

In [ ]:
import torch
from transformers import (
    AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,
    HfArgumentParser, TrainingArguments, pipeline, logging
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer, SFTConfig

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

In [ ]:
base_model_name = "meta-llama/Meta-Llama-3.1-8B"
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=bnb_config,
    device_map="auto"
)
base_model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_size = "right"

In [ ]:
# Cấu hình LoRA
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
sft_config = SFTConfig(
    output_dir="./sft-model-book-price",
    num_train_epochs=1,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="cosine",
    gradient_checkpointing=False,
    save_strategy="no",

    # riêng cho SFTTrainer
    # max_seq_length=1024,
    packing=False,
)

In [ ]:
def formatting_func(example):
    return f"User: {example['text']}\nAssistant: {example['price']}"

In [ ]:
formatting_func(train[0])

In [ ]:
trainer = SFTTrainer(
    model=base_model,
    processing_class=tokenizer,
    train_dataset=train,
    peft_config=peft_config,
    formatting_func=formatting_func,
    args=sft_config,
    eval_dataset=test
)

In [ ]:
trainer.train()

In [ ]:
trainer.save_model()

In [ ]:
trainer.push_to_hub(
    model_name="vmt311/sft-model-book-price"
)

In [ ]:
!nvidia-smi